In [19]:
# 모듈 import

from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from mmdet.utils import get_device

import argparse
import wandb


In [20]:

# 명령줄 인자 파서 설정
parser = argparse.ArgumentParser(description="Model Training Script")
parser.add_argument('--pos', type=float, default=0.5, help='pos_iou_thr')
parser.add_argument('--neg', type=int, default=0.5, help='neg_iou_thr')
parser.add_argument('--min', type=float, default=0.3, help='min_pos_iou')
parser.add_argument('--score', type=float, default=0.1, help='score_thr')
args = parser.parse_args()

usage: ipykernel_launcher.py [-h] [--pos POS] [--neg NEG] [--min MIN]
                             [--score SCORE]
ipykernel_launcher.py: error: unrecognized arguments: --f=/data/ephemeral/home/.local/share/jupyter/runtime/kernel-v32b426a24677f99d441e069f4292feb6a5edb151a.json


SystemExit: 2

/opt/conda/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [2]:
print(f"pos_iou_thr: {args.pos}")
print(f"neg_iou_thr: {args.neg}")
print(f"min_pos_thr: {args.min}")
print(f"score_thr: {args.score}")

In [8]:
model_name = f"faster_rcnn_fpn_pos_iou_thr{args.pos}_neg_iou_thr{args.neg}_min_pos_iou{args.min}_score_thr{args.score}"

In [21]:
classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")

# config file 들고오기
#cfg = Config.fromfile('./configs/a_my_config/my_config.py')
cfg = Config.fromfile('./configs/faster_rcnn/faster_rcnn_r50_fpn_1x_coco.py')
root='../../../../../dataset/'

# dataset config 수정
cfg.data.train.classes = classes
#cfg.data.train.palette = palette

cfg.data.train.img_prefix = root
cfg.data.train.ann_file = root + 'split_train.json' # train json 정보
cfg.data.train.pipeline[2]['img_scale'] = (512,512) # Resize

cfg.data.val.classes = classes
cfg.data.val.img_prefix = root
cfg.data.val.ann_file = root + 'split_val.json' # test json 정보
cfg.data.val.pipeline[1]['img_scale'] = (512,512) # Resize

cfg.data.test.classes = classes
cfg.data.test.img_prefix = root
cfg.data.test.ann_file = root + 'test.json' # test json 정보
cfg.data.test.pipeline[1]['img_scale'] = (512,512) # Resize

cfg.data.samples_per_gpu = 2
cfg.data.workers_per_gpu = 2

cfg.seed = 20
cfg.gpu_ids = [0]
cfg.work_dir = './work_dirs/'+ model_name

cfg.model.roi_head.bbox_head.num_classes = 10

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.checkpoint_config = dict(max_keep_ckpts=3, interval=1)
cfg.device = get_device()

cfg.runner.max_epochs = 30
cfg.workflow = [('train', 1)]
cfg.runner.meta = dict(exp_name = "exp_name")

In [ ]:
cfg.model.train_cfg.rpn.assigner.pos_iou_thr = args.pos
cfg.model.train_cfg.rpn.assigner.neg_iou_thr = args.neg
cfg.model.train_cfg.rpn.assigner.min_pos_iou = args.min
cfg.model.train_cfg.rcnn.assigner.pos_iou_thr = args.pos
cfg.model.train_cfg.rcnn.assigner.neg_iou_thr = args.neg
cfg.model.train_cfg.rcnn.assigner.min_pos_iou =args.neg
cfg.model.test_cfg.rcnn.score_thr = args.score

In [11]:
print(cfg.model.train_cfg.rpn)
print(cfg.model.train_cfg.rcnn)
print(cfg.model.test_cfg.rcnn)

In [12]:
wandb.login()

cfg.log_config.hooks = [
    dict(type='TextLoggerHook'),
    dict(type='MyMMDetWandbHook',
         init_kwargs={'project' : 'object-detection',
                      'name' : model_name},
         interval=10,
         log_checkpoint=True,
         log_checkpoint_metadata=True,
         num_eval_images=100
         )
         ]


wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


wandb: Currently logged in as: kaeh3403 (kaeh3403-personal). Use `wandb login --relogin` to force relogin


In [13]:
# build_dataset
datasets = [build_dataset(cfg.data.train)]
#valid_datasets = []

loading annotations into memory...
Done (t=0.06s)
creating index...
index created!


In [14]:
# dataset 확인
datasets[0]


CocoDataset Train dataset with number of images 3906, and instance counts: 
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
| category          | count | category      | count | category        | count | category    | count | category     | count |
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
| 0 [General trash] | 3179  | 1 [Paper]     | 5213  | 2 [Paper pack]  | 707   | 3 [Metal]   | 764   | 4 [Glass]    | 727   |
| 5 [Plastic]       | 2375  | 6 [Styrofoam] | 966   | 7 [Plastic bag] | 4114  | 8 [Battery] | 138   | 9 [Clothing] | 374   |
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+

In [15]:
# 모델 build 및 pretrained network 불러오기
model = build_detector(cfg.model)
model.init_weights()

2024-10-18 00:39:58,340 - mmcv - INFO - initialize ResNet with init_cfg {'type': 'Pretrained', 'checkpoint': 'torchvision://resnet50'}
2024-10-18 00:39:58,342 - mmcv - INFO - load model from: torchvision://resnet50
2024-10-18 00:39:58,343 - mmcv - INFO - load checkpoint from torchvision path: torchvision://resnet50
2024-10-18 00:39:58,455 - mmcv - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: fc.weight, fc.bias

2024-10-18 00:39:58,472 - mmcv - INFO - initialize FPN with init_cfg {'type': 'Xavier', 'layer': 'Conv2d', 'distribution': 'uniform'}
2024-10-18 00:39:58,499 - mmcv - INFO - initialize RPNHead with init_cfg {'type': 'Normal', 'layer': 'Conv2d', 'std': 0.01}
2024-10-18 00:39:58,505 - mmcv - INFO - initialize Shared2FCBBoxHead with init_cfg [{'type': 'Normal', 'std': 0.01, 'override': {'name': 'fc_cls'}}, {'type': 'Normal', 'std': 0.001, 'override': {'name': 'fc_reg'}}, {'type': 'Xavier', 'distribution': 'uniform', 'override'

In [16]:
cfg.runner.max_epochs = 20

In [18]:
# 모델 학습
train_detector(model, datasets[0], cfg, distributed=False, validate=True)

2024-10-18 01:13:57,900 - mmdet - INFO - Automatic scaling of learning rate (LR) has been disabled.


2024-10-18 01:13:57,926 - mmdet - INFO - Start running, host: root@instance-12243, work_dir: /data/ephemeral/home/KSG-git/level2-objectdetection-cv-20/code/baseline/mmdetection/work_dirs/faster_rcnn_fpn_pos_iou_thr{args.pos}_neg_iou_thr{args.neg}_min_pos_iou{args.min}_score_thr{args.score}
2024-10-18 01:13:57,927 - mmdet - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) CheckpointHook                     
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) MyMMDetWandbHook                   
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) NumClassCheckHook                  
(LOW         ) IterTimerHook                      
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) MyMMDetWandbHook                   
 ----

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


wandb: Downloading large artifact val:v0, 152.82MB. 102 files... 
wandb:   102 of 102 files downloaded.  
Done. 0:0:0.7
2024-10-18 01:15:00,312 - mmdet - INFO - Epoch [1][50/1953]	lr: 1.978e-03, eta: 0:04:53, time: 0.154, data_time: 0.048, memory: 1298, loss_rpn_cls: 0.0760, loss_rpn_bbox: 0.0409, loss_cls: 0.3194, acc: 92.0996, loss_bbox: 0.2219, loss: 0.6582, grad_norm: 1.4889
2024-10-18 01:15:05,884 - mmdet - INFO - Epoch [1][100/1953]	lr: 3.976e-03, eta: 0:04:05, time: 0.111, data_time: 0.004, memory: 1298, loss_rpn_cls: 0.0866, loss_rpn_bbox: 0.0388, loss_cls: 0.3369, acc: 91.5703, loss_bbox: 0.2319, loss: 0.6942, grad_norm: 1.7403
2024-10-18 01:15:11,286 - mmdet - INFO - Epoch [1][150/1953]	lr: 5.974e-03, eta: 0:03:44, time: 0.108, data_time: 0.004, memory: 1298, loss_rpn_cls: 0.0684, loss_rpn_bbox: 0.0385, loss_cls: 0.3237, acc: 91.4922, loss_bbox: 0.2441, loss: 0.6747, grad_norm: 1.6450
2024-10-18 01:15:16,772 - mmdet - INFO - Epoch [1][200/1953]	lr: 7.972e-03, eta: 0:03:31, ti

KeyboardInterrupt: 